[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-vector-generation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-vector-generation.ipynb)

# SPLADE Sparse-Dense Embedding Generation

## Overview

SPLADE is a class of models that produce sparse embeddings. Dense embeddings are often difficult to interpret, but sparse embeddings have clearly identifiable token overlap, making sparse vector search results more interpretable. SPLADE models have been shown to consistently outperform dense models, particularly in out-of-domain settings. 

The following guide will show you how to construct SPLADE embeddings to use in Pinecone's sparse-dense vectors. See the [companion guide to skip embedding generation](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb).

## Prerequisites

We'll install the required libraries:

In [ ]:
!pip install -qU \
          transformers \
          torch \
          sentence_transformers \
          tqdm \
          pandas

## Quora Dataset

We'll load the popular Quora dataset:

In [ ]:
import pandas as pd

df = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/quora_all-MiniLM-L6-v2_Splade/raw/quora_questions_sample200.parquet")

In [ ]:
df.head()

### Sparse Embeddings with SPLADE 

In the following example we will use the [naver/splade-cocondenser-ensembledistil](https://huggingface.co/naver/splade-cocondenser-ensembledistil) SPLADE model.


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

class SPLADE:
    def __init__(self, model):
        # check device
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.model = AutoModelForMaskedLM.from_pretrained(model)
        # move to gpu if available
        self.model.to(self.device)

    def __call__(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            logits = self.model(**inputs).logits

        inter = torch.log1p(torch.relu(logits[0]))
        token_max = torch.max(inter, dim=0)  # sum over input tokens
        nz_tokens = torch.where(token_max.values > 0)[0]
        nz_weights = token_max.values[nz_tokens]

        order = torch.sort(nz_weights, descending=True)
        nz_weights = nz_weights[order[1]]
        nz_tokens = nz_tokens[order[1]]
        return {
            'indices': nz_tokens.cpu().numpy().tolist(),
            'values': nz_weights.cpu().numpy().tolist()
        }

In [ ]:
splade = SPLADE("naver/splade-cocondenser-ensembledistil")

In [ ]:
doc = "what is the capital of france?"
sparse_vector = splade(doc)


### Dense Model

We use the popular all-MiniLM-L6-v2 model available on Hugging Face for dense vectors.

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"running on {device}")

model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)

### Compute Dense & Sparse Embeddings

Create BM25 sparse embeddings:

In [ ]:
from tqdm.notebook import tqdm

tqdm.pandas()
df['sparse_values'] = df['text'].progress_apply(lambda x: splade(x))

And now encode dense vector embeddings:

In [ ]:
df['values'] = df['text'].progress_apply(lambda x: model.encode(x))

We organize our dataframe to align to the `pinecone-datasets` format:

In [ ]:
df_result = df.copy()
df_result["metadata"] = None
df_result["blob"] = df_result["text"].apply(lambda t: {"text": t})
df_result = df_result.drop(columns="text")

In [ ]:
df_result.head()

And now we have all we need to start using Pinecone vector database 🚀

For more details on that, check out [this notebook](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb).